# Install and Import modules

In [1]:
# !tar xfvz /kaggle/input/ultralytics-for-offline-install/archive.tar.gz
# !pip install --no-index --find-links=./packages ultralytics
# !rm -rf ./packages

In [2]:
# !cp -r '/kaggle/input/hengck-czii-cryo-et-01/wheel_file' '/kaggle/working/'
# !pip install /kaggle/working/wheel_file/asciitree-0.3.3/asciitree-0.3.3
# !pip install --no-index --find-links=/kaggle/working/wheel_file zarr

In [3]:
import zarr
from ultralytics import YOLO
from tqdm import tqdm
import glob, os
import torch

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import cv2

In [5]:
DATA_KAGGLE_DIR = '../../raw'
EXP_NAME = "baseline_cv_syn_data_transfer_radius"
LIST_CV = [
    'TS_5_4',
    'TS_69_2',
    'TS_6_4',
    'TS_6_6',
    'TS_73_6',
    'TS_86_3',
    'TS_99_9',
]

In [6]:
WANDB = True
WANDB_EXP_NAME = f"{EXP_NAME}_optuna_best"
# EXP_NAME = "try"

if WANDB:
    # !pip install wandb
    import wandb
    import os
    from dotenv import load_dotenv
    load_dotenv()
    wandb.login(key=os.environ.get("WANDB_API_KEY"))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: trira7503 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


We use a recursive function in this notebook, and we change the settings to explore the graph deep enough.

In [7]:
import sys
sys.setrecursionlimit(10000)

In [8]:
import warnings
warnings.simplefilter('ignore')
np.warnings = warnings

# Prepare trained YOLO model

In [9]:
#add by @minfuka
assert torch.cuda.device_count() == 1

In [10]:
particle_names = ['apo-ferritin', 'beta-amylase', 'beta-galactosidase', 'ribosome', 'thyroglobulin', 'virus-like-particle']

In [11]:
p2i_dict = {
        'apo-ferritin': 0,
        'beta-amylase': 1,
        'beta-galactosidase': 2,
        'ribosome': 3,
        'thyroglobulin': 4,
        'virus-like-particle': 5
    }

i2p = {v:k for k, v in p2i_dict.items()}

In [12]:
dict_params = {
    'apo-ferritin': {
        'threshold_conf': 0.2866886276864816,
        'particle_radius': 65 * 0.3330259963871552,
        'particle_min_sample': 1,
        'particle_metric': "euclidean",
        'weighted_mean': 1,
        'threshold_total_conf': 0.29662649021298226,
    },
    'beta-amylase': {
        'threshold_conf': 999,
        'particle_radius': 65,
        'particle_min_sample': 999,
        'particle_metric': "euclidean",
        'weighted_mean': 999,
        'threshold_total_conf': 999,
    },
    'beta-galactosidase': {
        'threshold_conf': 0.03804188733411555,
        'particle_radius': 95 * 0.6125882701766436,
        'particle_min_sample': 2,
        'particle_metric': "manhattan",
        'weighted_mean': 0,
        'threshold_total_conf': 0.5481277130533789,
    },
    'ribosome': {
        'threshold_conf': 0.18266253469712418,
        'particle_radius': 150 * 0.36301986884673526,
        'particle_min_sample': 8,
        'particle_metric': "manhattan",
        'weighted_mean': 1,
        'threshold_total_conf': 1.5194529712970373,
    },
    'thyroglobulin': {
        'threshold_conf': 0.04784271636173772,
        'particle_radius': 135 * 0.5034473145901497,
        'particle_min_sample': 6,
        'particle_metric': "manhattan",
        'weighted_mean': 1,
        'threshold_total_conf': 0.5774365781570538,
    },
    'virus-like-particle': {
        'threshold_conf': 0.10075937641227564,
        'particle_radius': 145 * 1.4179581067925828,
        'particle_min_sample': 13,
        'particle_metric': "euclidean",
        'weighted_mean': 0,
        'threshold_total_conf': 0.37854066112353174,
    },
}

# define Main process class
There are many variables and functions going back and forth. We can easily manage variables by defining classes.

In [13]:
class PredAggForYOLO:
    def __init__(self, first_conf=0.2):
        self.first_conf = first_conf # threshold of confidence yolo

    def convert_to_8bit(self, x):
        lower, upper = np.percentile(x, (0.5, 99.5))
        x = np.clip(x, lower, upper)
        x = (x - x.min()) / (x.max() - x.min() + 1e-12) * 255
        return x.round().astype("uint8")

    # main routine.
    # change by @minfuka
    # def make_predict_yolo(self, r, model):
    def make_predict_yolo(self, r, model, device_no):
        vol = zarr.open(f'{valid_dir}/static/ExperimentRuns/{r}/VoxelSpacing10.000/denoised.zarr', mode='r')
        vol = vol[0]
        vol2 = self.convert_to_8bit(vol)
        n_imgs = vol2.shape[0]
    
        df = pd.DataFrame()
    
        pts = []
        confs = []
        xs = []
        ys = []
        zs = []
        
        for i in range(n_imgs):
            # Unfortunately the image size needs to be a multiple of 32.
            tmp_img = np.zeros((630, 630))
            tmp_img[:] = vol2[i]
    
            inp_arr = np.stack([tmp_img]*3,axis=-1)
            inp_arr = cv2.resize(inp_arr, (640,640))

            # change by @minfuka
            # res = model.predict(inp_arr, save=False, imgsz=640, conf=self.first_conf, device="0", batch=1, verbose=False)
            res = model.predict(inp_arr, save=False, imgsz=640, conf=self.first_conf, device=device_no, batch=1, verbose=False)
            for j, result in enumerate(res):
                boxes = result.boxes # Boxes object for bounding box outputs    
                for k in range(len(boxes.cls)):
                    ptype = i2p[boxes.cls.cpu().numpy()[k]] # particle type
                    conf = boxes.conf.cpu().numpy()[k] # confidence score
                    # YOLO can infer (start_x, end_x, start_y, end_y)
                    xc = (boxes.xyxy[k,0] + boxes.xyxy[k,2]) / 2.0 * 10 * (63/64)
                    yc = (boxes.xyxy[k,1] + boxes.xyxy[k,3]) / 2.0 * 10 * (63/64)
                    zc = i * 10 + 5
    
                    pts.append(ptype)
                    confs.append(conf)
                    xs.append(xc.cpu().numpy().item())  # numpy.float64 -> float
                    ys.append(yc.cpu().numpy().item())  # numpy.float64 -> float
                    zs.append(float(zc))  # 念のためfloatに変換         

        df['experiment'] = [r] * len(xs)
        df['particle_type'] = pts
        df['confidence'] = confs
        df['x'] = xs
        df['y'] = ys
        df['z'] = zs

        # df includes overall canditate of CIRCLE. 
        df = df.sort_values(['particle_type', 'z'], ascending=[True, True])

        final = []
        for pidx, p in enumerate(particle_names):
            if p == 'beta-amylase':
                continue
            params = dict_params[p]
            list_groups = []
    
            pdf = df[df['particle_type'] == p].reset_index(drop=True)
            pdf = pdf[pdf['confidence'] > params["threshold_conf"]].reset_index(drop=True)
            p_rad = params["particle_radius"]
    
            grouped = pdf.groupby(['experiment'])
    
            for exp, group in grouped:
                group = group.reset_index(drop=True)
    
                coords = group[['x', 'y', 'z']].values
                db = DBSCAN(
                    eps=p_rad,
                    min_samples=params["particle_min_sample"],
                    metric=params["particle_metric"]
                ).fit(coords)
                labels = db.labels_
    
                group['cluster'] = labels
    
                for cluster_id in np.unique(labels):
                    if cluster_id == -1:
                        continue
    
                    cluster_points = group[group['cluster'] == cluster_id]
    
                    if params["weighted_mean"]==1:
                        avg_x = (cluster_points['x'] * cluster_points['confidence']).sum() / cluster_points['confidence'].sum()
                        avg_y = (cluster_points['y'] * cluster_points['confidence']).sum() / cluster_points['confidence'].sum()
                        avg_z = (cluster_points['z'] * cluster_points['confidence']).sum() / cluster_points['confidence'].sum()
                    else:
                        avg_x = cluster_points['x'].mean()
                        avg_y = cluster_points['y'].mean()
                        avg_z = cluster_points['z'].mean()
                    total_conf = cluster_points['confidence'].sum()
    
                    group.loc[group['cluster'] == cluster_id, ['x', 'y', 'z', "total_conf"]] = avg_x, avg_y, avg_z, total_conf
                    # try:
                    #     group = group.drop_duplicates(subset=['x', 'y', 'z'])
                    # except TypeError as e:
                    #     print("Error detected in drop_duplicates:")
                    #     print(group[['x', 'y', 'z']].head())
                    #     print("Data types:\n", group[['x', 'y', 'z']].applymap(type))
                    #     raise e  # エラーを再度発生させて処理を止める
                    group = group.drop_duplicates(subset=['x', 'y', 'z'])            
    
                list_groups.append(group)
            # confidenceで調整
            tmp = pd.concat(list_groups, ignore_index=True)
            if "total_conf" in tmp.columns:
                tmp = tmp[tmp["total_conf"]>=params["threshold_total_conf"]]
                tmp.drop(columns=["total_conf", "confidence"], axis=1, inplace=True)
            final.append(tmp)
    
        submission = pd.concat(final, ignore_index=True)
        submission = submission.drop(columns=['cluster'])
        submission = submission.sort_values(by=['experiment', 'particle_type']).reset_index(drop=True)
        submission['id'] = np.arange(0, len(submission))
    
        return submission

In [14]:
# instance main class
agent = PredAggForYOLO(first_conf=0.03) # final_conf is not used after version 14

In [15]:
# subs = []

In [16]:
import time
#add by @minfuka
from concurrent.futures import ProcessPoolExecutor #add by @minfuka

# main loop of inference

In [17]:
valid_dir =f'{DATA_KAGGLE_DIR}/train'
list_model_path = [
    f"../../runs/detect/{EXP_NAME}/weights/best.pt",
    f"../../runs/detect/{EXP_NAME}2/weights/best.pt",
    f"../../runs/detect/{EXP_NAME}3/weights/best.pt",
    f"../../runs/detect/{EXP_NAME}4/weights/best.pt",
    f"../../runs/detect/{EXP_NAME}5/weights/best.pt",
    f"../../runs/detect/{EXP_NAME}6/weights/best.pt",
    f"../../runs/detect/{EXP_NAME}7/weights/best.pt",
]

In [18]:
#add by @minfuka
def inference(runs, model, device_no):
    subs = []
    for r in tqdm(runs, total=len(runs)):
        df = agent.make_predict_yolo(r, model, device_no)
        subs.append(df)
    
    return subs

In [19]:

# tick = time.time()
#change by @minfuka
subs = []
for r, model_path in tqdm(zip(LIST_CV, list_model_path), total=len(LIST_CV)):
    model = YOLO(model_path)
    df = agent.make_predict_yolo(r, model, "0")
    subs.append(df)
# with ProcessPoolExecutor(max_workers=2) as executor:
#     results = list(executor.map(inference, (runs1, runs2), (model, model), ("0", "1")))
# tock = time.time()

100%|██████████| 7/7 [01:58<00:00, 16.92s/it]


In [20]:
df

,experiment,particle_type,x,y,z,id
0,TS_99_9,apo-ferritin,3015.039981,336.148505,169.930764,0
1,TS_99_9,apo-ferritin,3080.836773,447.758153,180.370133,1
2,TS_99_9,apo-ferritin,3835.050386,206.353106,189.366127,2
3,TS_99_9,apo-ferritin,1285.993802,1752.317454,249.326059,3
4,TS_99_9,apo-ferritin,1238.365897,2054.780527,267.616678,4
...,...,...,...,...,...,...
361,TS_99_9,virus-like-particle,5689.547575,3347.511206,820.000000,361
362,TS_99_9,virus-like-particle,4204.167175,5535.808451,870.000000,362
363,TS_99_9,virus-like-particle,3017.341201,2470.538838,885.000000,363
364,TS_99_9,virus-like-particle,2249.352070,4311.831768,960.000000,364


In [21]:
#submission = pd.concat(subs).reset_index(drop=True)
#change by @minfuka
# submission1 = pd.concat(results[1])
# if len(valid_id) == 1:
#     submission = submission1.copy()
# else:
#     submission0 = pd.concat(results[0])
#     submission = pd.concat([submission0, submission1]).reset_index(drop=True)
submission = pd.concat(subs).reset_index(drop=True)
# submission.insert(0, 'id', range(len(submission)))

In [22]:
submission.head()

,experiment,particle_type,x,y,z,id
0,TS_5_4,apo-ferritin,5877.711650,5127.359857,87.329251,0
1,TS_5_4,apo-ferritin,5469.810989,1522.491265,85.025245,1
2,TS_5_4,apo-ferritin,5746.812174,5107.619718,100.763915,2
3,TS_5_4,apo-ferritin,5715.927478,4998.528522,123.457764,3
4,TS_5_4,apo-ferritin,5294.938844,4166.588778,144.757035,4


# Scoring

https://www.kaggle.com/code/hengck23/3d-unet-using-2d-image-encoder/notebook

In [23]:
import sys
sys.path.append('hengck')

from czii_helper import *
from dataset import *
from model2 import *
import numpy as np
from scipy.optimize import linear_sum_assignment

In [24]:
def do_one_eval(truth, predict, threshold):
    P=len(predict)
    T=len(truth)

    if P==0:
        hit=[[],[]]
        miss=np.arange(T).tolist()
        fp=[]
        metric = [P,T,len(hit[0]),len(miss),len(fp)]
        return hit, fp, miss, metric

    if T==0:
        hit=[[],[]]
        fp=np.arange(P).tolist()
        miss=[]
        metric = [P,T,len(hit[0]),len(miss),len(fp)]
        return hit, fp, miss, metric

    #---
    distance = predict.reshape(P,1,3)-truth.reshape(1,T,3)
    distance = distance**2
    distance = distance.sum(axis=2)
    distance = np.sqrt(distance)
    p_index, t_index = linear_sum_assignment(distance)

    valid = distance[p_index, t_index] <= threshold
    p_index = p_index[valid]
    t_index = t_index[valid]
    hit = [p_index.tolist(), t_index.tolist()]
    miss = np.arange(T)
    miss = miss[~np.isin(miss,t_index)].tolist()
    fp = np.arange(P)
    fp = fp[~np.isin(fp,p_index)].tolist()

    metric = [P,T,len(hit[0]),len(miss),len(fp)] #for lb metric F-beta copmutation
    return hit, fp, miss, metric


def compute_lb(submit_df, overlay_dir):
    valid_id = list(submit_df['experiment'].unique())
    print(valid_id)

    eval_df = []
    for id in valid_id:
        truth = read_one_truth(id, overlay_dir) #=f'{valid_dir}/overlay/ExperimentRuns')
        id_df = submit_df[submit_df['experiment'] == id]
        for p in PARTICLE:
            p = dotdict(p)
            # print('\r', id, p.name, end='', flush=True)
            xyz_truth = truth[p.name]
            xyz_predict = id_df[id_df['particle_type'] == p.name][['x', 'y', 'z']].values
            hit, fp, miss, metric = do_one_eval(xyz_truth, xyz_predict, p.radius* 0.5)
            eval_df.append(dotdict(
                id=id, particle_type=p.name,
                P=metric[0], T=metric[1], hit=metric[2], miss=metric[3], fp=metric[4],
            ))
    print('')
    eval_df_all = pd.DataFrame(eval_df)
    gb_all = []
    lb_score_all = []
    for exp in LIST_CV:
        eval_df = eval_df_all[eval_df_all['id'] == exp]
        gb = eval_df.groupby('particle_type').agg('sum').drop(columns=['id'])
        gb.loc[:, 'precision'] = gb['hit'] / gb['P']
        gb.loc[:, 'precision'] = gb['precision'].fillna(0)
        gb.loc[:, 'recall'] = gb['hit'] / gb['T']
        gb.loc[:, 'recall'] = gb['recall'].fillna(0)
        gb.loc[:, 'f-beta4'] = 17 * gb['precision'] * gb['recall'] / (16 * gb['precision'] + gb['recall'])
        gb.loc[:, 'f-beta4'] = gb['f-beta4'].fillna(0)

        gb = gb.sort_values('particle_type').reset_index(drop=False)
        # https://www.kaggle.com/competitions/czii-cryo-et-object-identification/discussion/544895
        gb.loc[:, 'weight'] = [1, 0, 2, 1, 2, 1]
        lb_score = (gb['f-beta4'] * gb['weight']).sum() / gb['weight'].sum()
        gb_all.append(gb)
        lb_score_all.append(lb_score)
    return gb_all, lb_score_all


def score_submission(submission):
    #if 1:
    submit_df=submission.copy()
    gb_all, lb_score_all = compute_lb(submit_df, '../../raw/train/overlay/ExperimentRuns')
    for gb, lb_score in zip(gb_all, lb_score_all):
        display(gb)
        print(f'lb_score: {lb_score:.4f}')
        print('')
        print("--------------------------------")

    return lb_score_all


    #show one ----------------------------------
    # fig = plt.figure(figsize=(18, 8))

    # id = valid_id[0]
    # truth = read_one_truth(id,overlay_dir=f'{valid_dir}/overlay/ExperimentRuns')

    # submit_df = submit_df[submit_df['experiment']==id]
    # for p in PARTICLE:
    #     p = dotdict(p)
    #     xyz_truth = truth[p.name]
    #     xyz_predict = submit_df[submit_df['particle_type']==p.name][['x','y','z']].values
    #     hit, fp, miss, _ = do_one_eval(xyz_truth, xyz_predict, p.radius)
    #     print(id, p.name)
    #     print('\t num truth   :',len(xyz_truth) )
    #     print('\t num predict :',len(xyz_predict) )
    #     print('\t num hit  :',len(hit[0]) )
    #     print('\t num fp   :',len(fp) )
    #     print('\t num miss :',len(miss) )

    #     ax = fig.add_subplot(2, 3, p.label, projection='3d')
    #     if hit[0]:
    #         pt = xyz_predict[hit[0]]
    #         ax.scatter(pt[:, 0], pt[:, 1], pt[:, 2], alpha=0.5, color='r')
    #         pt = xyz_truth[hit[1]]
    #         ax.scatter(pt[:,0], pt[:,1], pt[:,2], s=80, facecolors='none', edgecolors='r')
    #     if fp:
    #         pt = xyz_predict[fp]
    #         ax.scatter(pt[:, 0], pt[:, 1], pt[:, 2], alpha=1, color='k')
    #     if miss:
    #         pt = xyz_truth[miss]
    #         ax.scatter(pt[:, 0], pt[:, 1], pt[:, 2], s=160, alpha=1, facecolors='none', edgecolors='k')

    #     ax.set_title(f'{p.name} ({p.difficulty})')

    # plt.tight_layout()
    # plt.show()
    
    # #--- 
    # zz=0

In [25]:
lb_score_all = score_submission(submission)

['TS_5_4', 'TS_69_2', 'TS_6_4', 'TS_6_6', 'TS_73_6', 'TS_86_3', 'TS_99_9']



,particle_type,P,T,hit,miss,fp,precision,recall,f-beta4,weight
0,apo-ferritin,64,46,41,5,23,0.640625,0.891304,0.871250,1
1,beta-amylase,0,10,0,10,0,0.000000,0.000000,0.000000,0
2,beta-galactosidase,36,12,9,3,27,0.250000,0.750000,0.671053,2
3,ribosome,35,31,24,7,11,0.685714,0.774194,0.768362,1
4,thyroglobulin,92,30,25,5,67,0.271739,0.833333,0.743007,2
5,virus-like-particle,11,11,11,0,0,1.000000,1.000000,1.000000,1


lb_score: 0.7811

--------------------------------


,particle_type,P,T,hit,miss,fp,precision,recall,f-beta4,weight
0,apo-ferritin,60,35,35,0,25,0.583333,1.000000,0.959677,1
1,beta-amylase,0,12,0,12,0,0.000000,0.000000,0.000000,0
2,beta-galactosidase,41,16,11,5,30,0.268293,0.687500,0.629630,2
3,ribosome,42,37,34,3,8,0.809524,0.918919,0.911672,1
4,thyroglobulin,146,34,30,4,116,0.205479,0.882353,0.739130,2
5,virus-like-particle,9,9,9,0,0,1.000000,1.000000,1.000000,1


lb_score: 0.8013

--------------------------------


...

lb_score: 0.7825

--------------------------------


,particle_type,P,T,hit,miss,fp,precision,recall,f-beta4,weight
0,apo-ferritin,104,64,64,0,40,0.615385,1.000000,0.964539,1
1,beta-amylase,0,9,0,9,0,0.000000,0.000000,0.000000,0
2,beta-galactosidase,53,23,18,5,35,0.339623,0.782609,0.726841,2
3,ribosome,69,55,43,12,26,0.623188,0.781818,0.770285,1
4,thyroglobulin,149,45,37,8,112,0.248322,0.822222,0.723820,2
5,virus-like-particle,29,29,29,0,0,1.000000,1.000000,1.000000,1


lb_score: 0.8052

--------------------------------


,particle_type,P,T,hit,miss,fp,precision,recall,f-beta4,weight
0,apo-ferritin,63,36,36,0,27,0.571429,1.000000,0.957746,1
1,beta-amylase,0,21,0,21,0,0.000000,0.000000,0.000000,0
2,beta-galactosidase,64,24,16,8,48,0.250000,0.666667,0.607143,2
3,ribosome,75,65,56,9,19,0.746667,0.861538,0.853812,1
4,thyroglobulin,151,49,39,10,112,0.258278,0.795918,0.709091,2
5,virus-like-particle,13,13,13,0,0,1.000000,1.000000,1.000000,1


lb_score: 0.7777

--------------------------------


In [26]:
# wandbの初期化
if WANDB:
    wandb_config = {
        # ... 既存の設定 ...
        # "epochs": CONFIG['epochs'],
        # "learning_rate": CONFIG['learning_rate'],
        # "min_lr": CONFIG["min_lr"],
        # "weight_decay": CONFIG["weight_decay"],
        # "mixup_alpha": CONFIG["mixup_alpha"],
        # "mixup_epochs": CONFIG["mixup_epochs"],  # 新しく追加
    }
    wandb.init(project="CZII", name=WANDB_EXP_NAME, config=wandb_config)

for exp, score in zip(LIST_CV, lb_score_all):
    print(f'lb_score: {score:.4f}')
    if WANDB:
        wandb.log({f"lb_score_{exp}": score})
print(f'mean: {np.mean(lb_score_all):.4f}')
if WANDB:
    wandb.log({"mean_lb_score": np.mean(lb_score_all)})
    wandb.finish()



wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


lb_score: 0.7811
lb_score: 0.8013
lb_score: 0.7328
lb_score: 0.8057
lb_score: 0.7825
lb_score: 0.8052
lb_score: 0.7777
mean: 0.7837


lb_score_TS_5_4,▁
lb_score_TS_69_2,▁
lb_score_TS_6_4,▁
lb_score_TS_6_6,▁
lb_score_TS_73_6,▁
lb_score_TS_86_3,▁
lb_score_TS_99_9,▁
mean_lb_score,▁
lb_score_TS_5_4,0.7811
lb_score_TS_69_2,0.80127
lb_score_TS_6_4,0.73276


In [27]:
submission.to_csv(f"../../proc/sub/submission_{EXP_NAME}_{np.mean(lb_score_all):.4f}.csv", index=False)
